In [18]:
#import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
df = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
for i in range(5,10):
  print(df[r'review'][i])
  print("********\n")

Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it's not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother's slow awakening to what's happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they'd all be "up" for this movie.
********

I sure would like to see a resurrection of a up dated Seahunt series with the tech they have today it would bring back the kid excitement in me.I grew up on black and white TV and Seahunt with Gunsmoke were my hero's every week.You have my vote for a comeback of a new sea hunt.We need a change of pace in TV and this would work for 

In [7]:
df.shape

(50000, 2)

In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
#Data-Preprocessing

def remove_html_tags(text):

  clean = re.compile('<.*?>')
  return re.sub(clean,'',text)

In [15]:
df['review'] = df['review'].apply(remove_html_tags)

In [16]:
def remove_punctuations(text):

  re.sub(r'[^\w\s]','',text)
  return text

In [19]:
df['review'] = df['review'].apply(remove_punctuations)

In [21]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

mystopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
def remove_stopwords(text):

  text = [word.lower() for word in text.split() if word.lower() not in mystopwords]
  return " ".join(text)

In [26]:
df['review'] =  df['review'].apply(remove_stopwords)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer with default settings
vectorizer = CountVectorizer()

# Fit the vectorizer to the data and transform the text into a token matrix
token_matrix = vectorizer.fit_transform(df['review'])





In [29]:
df['review']

0        one reviewers mentioned watching 1 oz episode ...
1        wonderful little production. filming technique...
2        thought wonderful way spend time hot summer we...
3        basically there's family little boy (jake) thi...
4        petter mattei's "love time money" visually stu...
                               ...                        
49995    thought movie right good job. creative origina...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    catholic taught parochial elementary schools n...
49998    i'm going disagree previous comment side malti...
49999    one expects star trek movies high art, fans ex...
Name: review, Length: 50000, dtype: object

In [34]:
#Model Training
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(token_matrix, df['sentiment'], test_size=0.2, random_state=42)

In [38]:



from sklearn.linear_model import LogisticRegression

# Initialize and train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)




Accuracy: 0.8869


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [39]:
# Evaluate the model
accuracy = model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.8869


In [40]:
#Let's try decision tree classifier

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [41]:
#Let's evaluate

score = model.score(X_test,y_test)
print(score)

0.7344


In [42]:
predict = model.predict(X_test)
predict

array([1, 0, 0, ..., 1, 1, 1])

In [43]:
#let's try random forest


from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [44]:
score = model.score(X_test,y_test)
print(score)

0.86


In [45]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [46]:
score = model.score(X_test,y_test)
print(score)

0.8582


In [47]:
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_train, y_train)

BernoulliNB()

In [48]:
score= model.score(X_test,y_test)
print(score)

0.8546


In [51]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train.toarray(), y_train)

GaussianNB()

In [52]:
#Now we will use Gradient boosting classifier

from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)

GradientBoostingClassifier()

In [53]:
score = model.score(X_test,y_test)
print(score)

0.8126


In [55]:
#Let's try xgboost

import xgboost as xgb


dtrain = xgb.DMatrix(X_train, label=y_train)

dtest = xgb.DMatrix(X_test, label=y_test)

In [56]:
# Set parameters for XGBoost
params = {
    'objective': 'multi:softmax',  # Multiclass classification
    'num_class': len(np.unique(y_train)),  # Number of classes
    'eval_metric': 'merror'  # Evaluation metric
}

In [57]:
# Train XGBoost model
xgb_model = xgb.train(params, dtrain)

In [58]:
# Predict on the test set
y_pred_xgb = xgb_model.predict(dtest)

In [60]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred_xgb)
print(score)

0.7831


In [62]:
!pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 16.7 MB/s eta 0:00:00


In [63]:
from catboost import CatBoostClassifier

# Initialize CatBoost classifier
catboost_model = CatBoostClassifier()

In [64]:
# Train the classifier
catboost_model.fit(X_train, y_train)

# Predict on the test set
y_pred_catboost = catboost_model.predict(X_test)

# Calculate accuracy
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print("CatBoost Accuracy:", accuracy_catboost)

Learning rate set to 0.049775
0:	learn: 0.6785275	total: 307ms	remaining: 5m 6s
1:	learn: 0.6657057	total: 541ms	remaining: 4m 30s
2:	learn: 0.6532913	total: 776ms	remaining: 4m 17s
3:	learn: 0.6429770	total: 1.01s	remaining: 4m 12s
4:	learn: 0.6338264	total: 1.25s	remaining: 4m 8s
5:	learn: 0.6257367	total: 1.48s	remaining: 4m 5s
6:	learn: 0.6182079	total: 1.71s	remaining: 4m 2s
7:	learn: 0.6114263	total: 1.95s	remaining: 4m 1s
8:	learn: 0.6057340	total: 2.2s	remaining: 4m 2s
9:	learn: 0.5994693	total: 2.46s	remaining: 4m 3s
10:	learn: 0.5940500	total: 2.71s	remaining: 4m 3s
11:	learn: 0.5895977	total: 2.96s	remaining: 4m 3s
12:	learn: 0.5851336	total: 3.2s	remaining: 4m 2s
13:	learn: 0.5806286	total: 3.45s	remaining: 4m 2s
14:	learn: 0.5766448	total: 3.69s	remaining: 4m 2s
15:	learn: 0.5730051	total: 3.94s	remaining: 4m 2s
16:	learn: 0.5696248	total: 4.2s	remaining: 4m 2s
17:	learn: 0.5660673	total: 4.44s	remaining: 4m 2s
18:	learn: 0.5629100	total: 4.67s	remaining: 4m
19:	learn: 0.5

In [70]:
import lightgbm as lgb

# Convert sparse matrices to Dataset format used by LightGBM
# Convert label data to np.float32
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)


# Set parameters for LightGBM
params = {
    'objective': 'multiclass',
    'num_class': len(np.unique(y_train)),
    'metric': 'multi_error'
}

# Train LightGBM model
lgb_model = lgb.train(params, train_data, valid_sets=[test_data])

# Predict on the test set
y_pred_lgb = np.argmax(lgb_model.predict(X_test), axis=1)

# Calculate accuracy
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy:", accuracy_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.612480 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 59550
[LightGBM] [Info] Number of data points in the train set: 40000, number of used features: 15758
[LightGBM] [Info] Start training from score -0.691199
[LightGBM] [Info] Start training from score -0.695099
LightGBM Accuracy: 0.8616
